In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

from utils.config import Config
config = Config()
from utils.validation import Tensor

2025-06-20 22:21:54,409 - INFO - Current device: mps


## 1. Load data

- *Cargamos nuestros datos utilizando la función `torch.load()`. Utilizamos el modelo `Tensor` para validar nuestros tensores.*

In [12]:
# Load tensors
train_data = torch.load('temp/data/train_data.pth').to(device=config.device)
test_data = torch.load('temp/data/test_data.pth').to(device=config.device)

train_data = Tensor(tensor=train_data, tensor_dimensions=2)
test_data = Tensor(tensor=test_data, tensor_dimensions=2)

## 2. Build the model

- *Podemos crear nuestro modelo definiendo una capa oculta a la vez (como en la primera versión). Cuando definamos el método `forward()` vamos a tener que, explícitamente, definir el grafo computacional de nuestra red neuronal.*
- *Otra opción es utilizar el módulo `nn.Sequential`. Este módulo nos permite definir de forma implícita el grafo computacional de nuestra red neuronal.*
    - *Funciona como un contenedor en donde definimos los módulos (capas) de nuestra red neuronal en el orden que queremos que se ejecuten al llamar el método `forward()`.*
    - *Lo bueno es que no tenemos que definir el grafo computacional dentro del método `forward()`, lo que reduce considerablemente el código.* 
    - *Se puede encontrar más información en la [documentación](https://docs.pytorch.org/docs/stable/generated/torch.nn.Sequential.html).*

In [13]:
class NeuralNetworkV0(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, device: torch.device):
        super().__init__()
        self.layer_1 = nn.Linear(input_size, hidden_size)
        self.layer_2 = nn.Linear(hidden_size, 1)
        
        self.to(device)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        try:
            # Pass the input tensor through the first layer
            x = self.layer_1(x)
            # Apply the ReLU activation function
            x = torch.relu(x)
            # Pass the output of the first layer through the second layer
            x = self.layer_2(x)
            # Apply the sigmoid activation function for binary classification
            x = torch.sigmoid(x)

            return x
        except Exception as error:
            print(error)

class NeuralNetworkV0(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, device: torch.device):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        self.to(device)
    
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        try:
            return self.model(X)
        except Exception as error:
            print(error)

model = NeuralNetworkV0(
    input_size=train_data.tensor.shape[1],
    hidden_size=5,
    device=config.device
)

### 2.1. Architecture

⚠️ WIP: Explicación de la estructura de la red neuronal y los cálculos matemáticos que ocurren detrás.

<img src="attachments/simple-linear-nn.png" width="600" height="600" style="display: block; margin: 0 auto;" />

### 2.2. Loss function

- *Dado que este es un problema de clasificación binaria, vamos a utilizar la Binary Cross-Entropy como función de pérdida.*
- *PyTorch tiene disponibles dos clases para esta función: `torch.nn.BCELoss` y `torch.nn.BCELossWithLogits`. La única diferencia entre ambas es que `torch.nn.BCELossWithLogits` recibe los scores del modelo y con ellos computa las probabilidades (con la función de activación Sigmoidea) y la log-verosimilitud, mientras que `torch.nn.BCELoss` recibe directamente las probabilidades.*
- *Lo recomendable es utilizar `torch.nn.BCELossWithLogits` por dos razones:*
    1. *Nos ahorramos de definir la función de activación dentro de nuestro modelo.*
    2. *Computar las probabilidades (i.e., pasar los scores por la función de activación) y luego aplicar logaritmos suele ser una operación muy inestable numéricamente. Al calcular todo al mismo tiempo, reducimos las inestabilidades.*

In [14]:
criterion = nn.BCEWithLogitsLoss()

### 2.1. Optimizador

In [15]:
optimizer = optim.Adam(
    params=model.parameters(),
    lr=1e-3
)